In [ ]:
import pandas as pd, numpy as np, tensorflow as tf,  keras_nlp as nlp, keras
from sklearn.model_selection import train_test_split

In [1]:
#Generate split
generate_split = False
dev = True
if generate_split:
    df = pd.read_csv('./data/en-fr.csv', encoding='latin-1')
    train, test = train_test_split(df, test_size=0.2)
    train, test = pd.DataFrame(train), pd.DataFrame(test) #My LSP is bugged, this shut up the warnings
    train.to_csv('./data/train.csv', index=False)
    train.iloc[:10000].to_csv('./data/smol_train.csv', index=False)
    test.to_csv('./data/test.csv', index=False)
    test.iloc[:1000].to_csv('./data/smol_test.csv', index=False)
    if dev:
        train = train.iloc[:10000]
        test = test.iloc[:1000]
else:
    if dev:
        train = pd.read_csv('./data/smol_train.csv', encoding='latin-1')
        test = pd.read_csv('./data/smol_test.csv', encoding='latin-1')
    else:
        train = pd.read_csv('./data/train.csv', encoding='latin-1')
        test = pd.read_csv('./data/test.csv', encoding='latin-1')
print(train.shape, test.shape)
print(train.head())
print(test.head())

(10000, 2) (1000, 2)
                                                         en                                                 fr
13092249  This is accomplished through a range of activi...  Toute une gamme d'activitÃ©s lui permet d'atte...
21514116  Ms. Sabir was understandably very offended by ...  Mme Sabir a tout naturellement Ã©tÃ© offensÃ©e...
1883214   She will not report the interest income from h...  Elle n'a pas Ã  inclure dans sa dÃ©claration c...
16979510  On the part of Mr. Morken, while he may first ...  En ce qui concerne M. Morken, mÃªme si, au dÃ©...
2643386   E Jean-Pierre Kingsley Chief Electoral Officer...  Ã Jean-Pierre Kingsley Directeur gÃ©nÃ©ral de...
                                                         en                                                 fr
18826918  FinTRAC will also have primary responsibility ...  Le Centre aura aussi la responsabilitÃ© de sâ...
15308515   It is required to report to Parliament annually.  Maintien de l'autonomie financ

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[2], line 1
----> 1 train, test = train_test_split(df, test_size=0.2)

NameError: name 'train_test_split' is not defined


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[3], line 1
----> 1 train

NameError: name 'train' is not defined


In [ ]:
# Examples from kerasnlp
# Preprocessing params.
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

# Model params.
NUM_LAYERS = 3
MODEL_DIM = 256
INTERMEDIATE_DIM = 512
NUM_HEADS = 4
DROPOUT = 0.1
NORM_EPSILON = 1e-5

# Training params.
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3

In [ ]:
data = tf.data.experimental.CsvDataset('./data/en-fr.csv', [tf.string, tf.string], header=True).batch(FINETUNING_BATCH_SIZE) 

In [4]:
print(data.unbatch().batch(4).take(1).get_single_element())

(<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'Changing Lives | Changing Society | How It Works | Technology Drives Change Home | Concepts | Teachers | Search | Overview | Credits | HHCC Web | Reference | Feedback Virtual Museum of Canada Home Page',
       b'Site map', b'Feedback', b'Credits'], dtype=object)>, <tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'Il a transform\xc3\xa9 notre vie | Il a transform\xc3\xa9 la soci\xc3\xa9t\xc3\xa9 | Son fonctionnement | La technologie, moteur du changement Accueil | Concepts | Enseignants | Recherche | Aper\xc3\xa7u | Collaborateurs | Web HHCC | Ressources | Commentaires Mus\xc3\xa9e virtuel du Canada',
       b'Plan du site', b'R\xc3\xa9troaction', b'Cr\xc3\xa9dits'],
      dtype=object)>)


In [ ]:
tokenizer = nlp.tokenizers.WordPieceTokenizer()